In [168]:
# from attr import dataclass
from dataclasses import dataclass, field
from email import header
from logging import config
from operator import index
from sqlmodel import values
import swtoolkit as swtk
import win32com.client
import os
import csv
import pythoncom
import pywintypes
import openpyxl





def check_file_exists(file_path):
    """Checks if a file exists. Returns True if it does, False if it doesn't."""
    if os.path.isfile(file_path):
        return 
    else:
        open(file_path, "w").close



def get_design_table_from_model_as_list(sw_model):
    """Returns the design table from a model. Returns None if no design table exists."""
    design_table = sw_model.GetDesignTable

    if design_table is None:
        return None
    else:
        dt = []
        nTotRow = design_table.GetTotalRowCount
        nTotCol = design_table.GetTotalColumnCount
        nTotRow = nTotRow + 1
        nTotCol = nTotCol + 1


        design_table.Attach

        # Generate Header Row
        header_row = []
        header_row.append("Config_Name")

        for c_index in range(1, nTotCol):
            header_row.append(design_table.GetHeaderText(c_index-1))

        dt.append(header_row)

        # Populate List of table row data
        for r_index in range(1, nTotRow):
            row_data = []
            for c_index in range(nTotCol):
                row_data.append(design_table.GetEntryValue(r_index, c_index))
            dt.append(row_data)    
        
        design_table.Detach

        return dt


@dataclass
class SW_Configuration:
    """Class for Solidworks Configurations - represents a row in a design table"""
    config_name: str
    header_columns: list
    row_data: list

    def update_config(self):
        """Updates the configuration values in Solidworks Design Table"""
        pass

    def add_config(self):
        """Adds a new configuration to the Solidworks Design Table"""
        pass


@dataclass
class SW_DesignTable:
    """Class for Solidworks Design Tables"""
    sw_model: win32com.client.CDispatch
    sw_template_dir: str 
    name: str = field(init=False)
    table: list = field(init=False)

    def __post_init__(self):
        self.name = self.sw_model.GetTitle
        self.table = self.get_configuration_table_from_model_as_list()

    def get_design_table_from_model_as_list(self):
        """Returns the design table from a model. Returns None if no design table exists."""
        design_table = self.sw_model.GetDesignTable
        if design_table is None:
            return None
        else:
            bool = design_table.Attach

            # Generate Header Row
            header_row = []
            header_row.append("Config_Name")
            nTotRow = design_table.GetTotalRowCount + 1
            nTotCol = design_table.GetTotalColumnCount + 1
            for c_index in range(1, nTotCol):
                header_row.append(design_table.GetHeaderText(c_index-1))
            print(f"Header Row: {header_row}")
                
            print(f"Table Data:")
            for r_index in range( nTotRow):
                row_data = []
                for c_index in range(nTotCol):
                    print(f"      Row: {r_index} | Col: {c_index} | Value: {design_table.GetEntryText(r_index, c_index)}")
                    row_data.append(design_table.GetEntryText(r_index, c_index))
                print(f"      Row Data: {row_data}")
                # configs.append(SW_Configuration(row_data[0], header_row, row_data[1:]))  

            # Populate List of SW_Configuration objects
            configs = []
            for r_index in range(1, nTotRow):
                row_data = []
                for c_index in range(nTotCol):
                    row_data.append(design_table.GetEntryText(r_index, c_index))
                print(f"      Row Data: {row_data}")
                configs.append(SW_Configuration(row_data[0], header_row, row_data[1:]))    

            bool = design_table.Detach
            return configs

    def get_configuration_table_from_model_as_list(self):
        config_names = self.sw_model.GetConfigurationNames
        config_mgr = self.sw_model.ConfigurationManager
        print(f"\nConfig Names: {config_names}")
        parameters = []
        config_table = []
        config_obj_table = []
        for config_name in config_names:
            config = self.sw_model.GetConfigurationByName(config_name)
            config_table.append(config)
        print(f"Config Table: {config_table}\n")
        for config in config_table:
            print(f"   {config.Name}")
            print(f"   {config.Description}")
            num_params = config_mgr.GetConfigurationParamsCount(config.Name)
            print(f"   Number of Params = {num_params}")
            # ParameterNames = []
            # for i in range(num_params):
            #     ParameterNames.append("")
            # ParameterNames = win32com.client.VARIANT(pythoncom.VT_ARRAY | pythoncom.VT_BSTR, ParameterNames)
            ParameterNames = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
            # PropTypes = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
            ParameterValues = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
            # ParameterValues = []
            # for i in range(num_params):
            #     ParameterValues.append("")

            # ParameterValues = win32com.client.VARIANT(pythoncom.VT_ARRAY | pythoncom.VT_BSTR, ParameterValues)

            boolret = config_mgr.GetConfigurationParams(config.Name, ParameterNames, ParameterValues)
            print(f"   get config params: {boolret}")
            print(f"   {ParameterNames.value}")
            
            print(f"   {ParameterValues.value}")
            config_obj = SW_Configuration(config.Name, ParameterNames.value, ParameterValues.value)
            print(f"   {config_obj}")
            config_obj_table.append(config_obj)
            # for param in ParameterNames:
                # print(f"      {param}")
            # print(f"   {config.Panel_Sketch}")
            # param, val = config.GetParameters()
            # print(f"   {param} | {val}")
            # print(f"   {config.GetParameterCount}")
            # param, val = config.GetParameters
            # names = [""] * 7
            # values = [""] * 7
            # param = config.GetParameters(names, values)

            # print(f"   Param | Val  = {len(param)}") # | {val}")
            # print(f"   {config.GetDescription}")
            # print(f"   {config.GetParameters}")
        return config_obj_table

    def update_table(self):
        """Updates the configuration values in Solidworks Config Table"""
        # Iterate through each configuration in the table, then through each parameter in the configuration
        config_names = self.sw_model.GetConfigurationNames
        part = sw._active_doc()

        for config_row in self.table:
            for config_name in config_names:
                if config_name == config_row.config_name:
                    boolstatus = part.ShowConfiguration2(config_name)
                    print(f"Config Name: {config_name} | Status: {boolstatus}")
                    count = 0
                    for index, param in enumerate(config_row.header_columns):
                        # if param == "Config_Name":
                        if index < 2:
                            continue
                        else:
                            print(f"   Param | RowData = {param} | {config_row.row_data[index]} ")
                            parameter_val = part.Parameter(param) #, config_row.parameter_values[param])
                            print(f"   {parameter_val.Name} | {parameter_val.SystemValue}")
                            # print(f"   current value: {parameter_val.SystemValue} | new value: {config_row.parameter_values[index]}")
                            print(f"    {param} | {config_row.row_data[index]} ")
                            parameter_val.SystemValue = float(config_row.row_data[index]) / 1000 * 25.4
                            print(f"   updated value: {parameter_val.SystemValue} ")
                            print(f"                = {parameter_val.SystemValue * 1000 / 25.4}")
                            # print(f"   Param: {param} | Status: {boolstatus}")
                    # config = self.sw_model.GetConfigurationByName(config_name)
                    # for col_index in range(len(config_row.header_columns)):
                    #     if config_row.header_columns[col_index] == "Config_Name":
                    #         continue
                    #     else:
                    #         config.SetConfigurationParameter(config_row.header_columns[col_index], config_row.parameter_values[col_index])
        part.EditRebuild3

        # # design_table = self.sw_model.GetDesignTable
        # # design_table.Attach
        # config_mgr = self.sw_model.ConfigurationManager
        # config_names = self.sw_model.GetConfigurationNames
        # active_config = config_mgr.ActiveConfiguration

        # for config_row in self.table:
        #     for config_name in config_names:
        #         print(f"Config Name: {config_name}")
        #         if config_name == config_row.config_name:
        #             config = self.sw_model.GetConfigurationByName(config_name)

        #             print(f"\nConfig Name Matched on: {config_row.config_name}\n")
        #             print(f"Config: {config.Name}")
        #             print(f"checking config row data types...\n")
        #             col_count = 0
        #             header_row = config_row.header_columns #[2:]
        #             data_row = [] # config_row.row_data[2:]

        #             for param in config_row.row_data: #[2:]:
        #                 # col_count += 1
            
        #                 print(f"      Val #{col_count}: {param} | {type(param)}")
        #                 # if col_count != 1:
        #                 # param = float(param)
        #                 data_row.append(param)

        #             print(f"\n\nDid type conversion work...\n\n")
        #             for item in data_row:
        #                 print(f"      {item} | {type(item)}")

        #             param_names = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, header_row)
        #             param_values = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, data_row)
                    
        #             print(f"\nParam Names | type: {param_names.value} | {param_names.varianttype}")
        #             print(f"Param Value | type: {param_values.value} | {param_values.varianttype}")
        #             # retVal2 = config_mgr.SetConfigurationParams(config_name, param_names, param_values)
        #             select_bool = config.Select2(False, pythoncom.Nothing)
        #             print(f"    Select Bool: {select_bool}")
        #             config.SetParameters(param_names, param_values)
        #             # print(f"Update Table Config Params successful? {retVal2}")
        #             # print(f"\n\nTrying the IsetParams...\n\nConfig: {config_row}")
        #             # num_params = config_mgr.GetConfigurationParamsCount(config_row.config_name)
        #             # param_names = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_BSTR, config_row.header_columns)
        #             # param_values = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_BSTR, config_row.row_data)
        #             # print(f"Config Name: {config_row.config_name}")
        #             # print(f"Num Params: {num_params}")
        #             # print(f"Param Names | type: {param_names.value} | {param_names.varianttype}")
        #             # print(f"Param Value | type: {param_values.value} | {param_values.varianttype}")
        #             # retVal2 = config_mgr.ISetConfigurationParams(config_row.config_name, int(num_params), param_names, param_values)
        #             # print(f"Set Config Params 2 successful? {retVal2}")

        # # for r_idx, config_row in enumerate(self.table):
        # #     # Get the configuration name from the design table
        # #     sw_config_name, is_text = design_table.GetEntryValue(r_idx, 0)
        # #     # If the configuration names match, update the row
        # #     if sw_config_name == config_row.config_name:
        # #         for c_idx, value in enumerate(config_row.row_data):
        # #             is_text = isinstance(value, str)
        # #             design_table.SetEntryValue(r_idx, c_idx+1, is_text, value)  # c_idx+1 because we're starting from second column

        # # design_table.UpdateTable(2, True)  # 2 corresponds to swUpdateDesignTableAll constant
        # # design_table.Detach

    def add_config(self, config_row: SW_Configuration):
        """Adds a new configuration to the Solidworks Configuration Table"""
        config_names = self.sw_model.GetConfigurationNames
        part = sw._active_doc()
        config_mgr = self.sw_model.ConfigurationManager
        config_Param_count = config_mgr.GetConfigurationParamsCount(config_row.config_name)
        print(f"Number of Parameters: {config_Param_count}")
        config_exists = False
        # ParameterNames = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
        # PropTypes = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
        # ParameterValues = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
        # act_config = config_mgr.ActiveConfiguration
        # act_config.GetParameters(ParameterNames, ParameterValues)
        # print(f"Config: {act_config.Name}\n  Names:\n    {ParameterNames.value}\n  Values:\n    {ParameterValues.value}")
        
        # for config in config_mgr:
        #     ParameterNames = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
        #     # PropTypes = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
        #     ParameterValues = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, '')
        #     # act_config = config_mgr.ActiveConfiguration
        #     config.GetParameters(ParameterNames, ParameterValues)
        #     print(f"Config: {config.Name}\n  Names:\n    {ParameterNames.value}\n  Values:\n    {ParameterValues.value}")
        for config in config_names:

            if config == config_row.config_name:
                config_exists = True
                # raise ValueError(f"Configuration {config} already exists in model")
        if not config_exists:
            print(f"Adding Configuration: {config_row.config_name}")
            new_config = config_mgr.AddConfiguration(config_row.config_name, "", "", 128,"","")
            print(f"New Config | Type: {new_config} | {type(new_config)}")
            # self.update_table()
            # varNone = win32com.client.VARIANT(pythoncom.VT_EMPTY, None)
            param_names = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, config_row.header_columns)
            param_values = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, config_row.row_data)
            param_namesI = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_BSTR, config_row.header_columns)
            param_valuesI = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_BSTR, config_row.row_data)
            # param_names = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, "")
            # param_values = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_VARIANT, "")
            # print(f"IParam Names | type: {param_namesI.value} | {param_namesI.varianttype}")
            # print(f"IParam Value | type: {param_valuesI.value} | {param_valuesI.varianttype}")
            # select_bool = new_config.Select2(False, pythoncom.Nothing)
            # print(f"    Select Bool: {select_bool}")
            # new_config.SetParameters(param_names, param_values)
            # retVal = config_mgr.ISetConfigurationParams(config_row.config_name, config_Param_count, param_namesI, param_valuesI)
            # print(f"Set Config Params 1 successful? {retVal}")
            print(f"Param Names | type: {param_names.value} | {param_names.varianttype}")
            print(f"Param Value | type: {param_values.value} | {param_values.varianttype}")
            retVal2 = config_mgr.SetConfigurationParams(str(config_row.config_name), param_names, param_values)
            part.EditRebuild3
            print(f"Add new Config Params successful? {retVal2}")
            # print(f"Set Config Params successful? {retVal}")
        
        # # Design Table Method:
        # design_table = self.sw_model.GetDesignTable
        # num_cols = design_table.GetTotalColumnCount
        # num_cols = num_cols + 1
        # # empty_col_array = 
        # # cell_array = win32com.client.VARIANT(pythoncom.VT_ARRAY | pythoncom.VT_BSTR, [])
        # design_table.Attach
        # cells = [config_row.config_name] + config_row.row_data
        # # cells = config_row.row_data
        # print(f"\n\n{cells}\n\n")
        # cell_array = []
        # for index, cell in enumerate(cells):
        #     print(f"      Cell | Type: {cell} | {type(cell)}")
        #     cell_array.append(cell)
        # cell_array = win32com.client.VARIANT(pythoncom.VT_BYREF | pythoncom.VT_BSTR, cell_array)

        # # Here is an example of creating a VARIANT array
        # print(f"      New Row Cells: {cells} | {type(cells)}")
        # print(f"      New Row Cells: {cell_array} | {type(cell_array)}")
        # worksheet = design_table.EditTable
        # print(f"      Worksheet: {worksheet}")
        # # boolstatus = win32com.client.VARIANT(pythoncom.VT_BOOL, design_table.AddRow(cell_array))
        # print(cell_array.value)
        # boolstatus = design_table.AddRow(cell_array)
        # print(f"    AddRow successful?: {boolstatus}")
        # boolstatus = design_table.UpdateTable(2, True)  # 2 corresponds to swUpdateDesignTableAll constant
        # print(f"    UpdateTable successful?: {boolstatus}")
        # design_table.Detach

    def write_to_csv(self):
        """Writes the design table to a csv file with the same name as the model. Creates new CSV if doesnt exist, completely overwrites if it does."""
        csv_file_path = os.path.join(self.sw_template_dir, self.name + ".csv")
        header = ["Configuration_Name"] + list(self.table[0].header_columns)
        print(f"    Writing header: {header} | type: {type(header)}")
        try:
            with open(csv_file_path, 'w') as csv_file:
                # Write the header row
                csv_file.write(",".join(header) + "\n")
                for config in self.table:
                    print(f"Writing config {config.config_name} to CSV file...")
                    print(f"    config.row_data: {config.row_data}")
                    row = [config.config_name] + list(config.row_data)
                    print(f"    Write Row: {row}")
                    # print(f"    config.row_data: {config.row_data}")
                    csv_file.write(",".join(row) + "\n")
        except FileNotFoundError:
            open(csv_file_path, "w").close()

    def update_from_csv(self):
        """Updates the design table from a CSV file"""
        print(f"\n\nUpdating design table from CSV file...\n")
        # Build the path to the CSV file
        csv_file_path = os.path.join(self.sw_template_dir, self.name + ".csv")

        # Read the CSV file
        with open(csv_file_path, 'r') as csv_file:
            reader = csv.reader(csv_file)
            header_row = next(reader)
            print(f"  Header Row: {header_row}")
            csv_configurations = []

            for row in reader:
                # Create a SW_Configuration for each row in the CSV file
                print(f"Config: {row[0]} - {header_row[1:]} - {row[1:]}")
                csv_configurations.append(SW_Configuration(row[0], header_row[1:], row[1:]))
                print(f"  CSV Config: {csv_configurations[-1]}")
        for config in csv_configurations:
            print(f"CSV Config: {config}")
        # Iterate over the configurations from the CSV file
        print(f"\n\nTable Object: {self.table}\n\n")
        for csv_config in csv_configurations:
            # Check if this configuration is already in the design table
            in_table = False
            for dt_config in self.table:
                print(f"  DT Config: {dt_config}")
                print(f"  Comparing {dt_config.config_name} to {csv_config.config_name}")
                if dt_config.config_name == csv_config.config_name:
                    in_table = True
                    # The configuration is in the design table, so update it
                    dt_config.row_data = csv_config.row_data
                    # self.update_config(dt_config)
                    # break
                # else:
                #     dt_config.row_data = csv_config.row_data
            if not in_table:
                # The configuration is not in the design table, so add it
                print(f"  Adding {csv_config} to design table...")
                # dt_config.row_data = csv_config.row_data
                self.add_config(csv_config)
        self.update_table()

    def update_csv_from_excel(self):
        """Updates the CSV file from an Excel file"""
        print(f"\n\nUpdating CSV file from Excel file...\n")
        # Build the path to the CSV file
        header_row = ["Configuration_Name"] + list(self.table[0].header_columns)
        print(f"  Header Row for updating csv: {header_row}")
        excel_template_dir = r'C:\Users\alaureijs\OneDrive - moorecoinc\Projects\Customs\Custom_Configurator\Array'
        csv_file_path = os.path.join(self.sw_template_dir, self.name + ".csv")
        # Build the path to the Excel file
        excel_file_path = os.path.join(excel_template_dir, self.name + "-Web.xlsx")
        # Read the Excel file
        Xlsx = win32com.client.DispatchEx('Excel.Application')
        Xlsx.DisplayAlerts = False
        Xlsx.Visible = False
        book = Xlsx.Workbooks.Open(excel_file_path)
        # Refresh my two sheets
        book.RefreshAll()
        Xlsx.CalculateUntilAsyncQueriesDone()
        book.Save()
        book.Close()
        Xlsx.Quit()
        # xl = openpyxl
        # wkbk = xl.open(excel_file_path, data_only=False)
        
        wb = openpyxl.load_workbook(excel_file_path, data_only=True)
        
        ws = wb.active
        # Read Excel file into a list of lists
        excel_configurations = []
        for row in ws.iter_rows(min_row=2, values_only=True):
            return_row = []
            for cell in row:
                # print(f"  Cell: {cell} | {type(cell)}")
                # if cell.lstrip("=") = 
                return_row.append(str(cell))
            excel_configurations.append(return_row)
        # Print the Excel configurations
        for config in excel_configurations:
            print(f"  Excel Config: {config}")
        # print(f"  Excel Configurations: {excel_configurations}")
        # 
        # Write to CSV file
        with open(csv_file_path, 'w') as csv_file:
            # Write the header row
            print(f"  Writing header: {header_row}")
            csv_file.write(",".join(header_row) + "\n")
            # Write each configuration
            for config in excel_configurations[1:]:
                print(f"  Writing config: {config}")
                csv_file.write(",".join(config) + "\n")
        # Update the design table from the CSV file
        # self.update_from_csv()
        return
            


sw = swtk.SolidWorks()

# get active document
model = sw._active_doc()

# sw.RunMacro2(path, moduleName, procName, 1, 0) # swRunMacroOption_e.swRunMacroUnloadAfterRun, 0)

dt = SW_DesignTable(model, r'G:\My Drive\Google Drive - Work\TestMacros')

dt.update_csv_from_excel()
dt.update_from_csv()

# config_list = dt.get_configuration_table_from_model_as_list()
print(f"\n\nUpdating design table from CSV file...\n")
# dt.update_from_csv()
# print(f"\n\nConfig List: {config_list}\n\n")
# sw_model = sw.get_model()
# sw_model.extension.rebuild(1)
print(f"\n\nDone.\n\n")




Config Names: ('Default', '60Hx30Wx12D', '40Hx40Wx20D', '48Hx24Wx12D', '24Hx60Wx24D', '80Hx60Wx36D', '120Hx48Wx48D', '30Hx30Wx30D')
Config Table: [<COMObject <unknown>>, <COMObject <unknown>>, <COMObject <unknown>>, <COMObject <unknown>>, <COMObject <unknown>>, <COMObject <unknown>>, <COMObject <unknown>>, <COMObject <unknown>>]

   Default
   Default
   Number of Params = 5
   get config params: True
   ('$DESCRIPTION', '$COLOR', 'Overall_Height@Cabinet_Height', 'Half_Width@Cabinet_Width', 'Overall_Depth@Cabinet_Depth')
   ('Default', '15651274.000000', '60.000000', '30.000000', '12.000000')
   SW_Configuration(config_name='Default', header_columns=('$DESCRIPTION', '$COLOR', 'Overall_Height@Cabinet_Height', 'Half_Width@Cabinet_Width', 'Overall_Depth@Cabinet_Depth'), row_data=('Default', '15651274.000000', '60.000000', '30.000000', '12.000000'))
   60Hx30Wx12D
   60Hx30Wx12D
   Number of Params = 5
   get config params: True
   ('$DESCRIPTION', '$COLOR', 'Overall_Height@Cabinet_Height